In [2]:
import os
import pandas as pd
import datetime as dt
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM
from keras.layers import Dropout
from pandas import Series
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
etf_list = ['SPY','IVV','VTI','VOO','QQQ','VEA','EFA','IEFA','VWO','AGG','IJH','IEMG','IWM','IJR','VTV','IWF','IWD','VUG','BND','LQD','XLF','VNQ','VIG','EEM','GLD','VB','BSV','VO','TIP','VEU','IVW','DIA','XLK','VYM','VGT','VCSH','MDY','IWB','VCIT','XLV','IWR','XLE','DVY','USMV','EWJ','VGK','PFF','SCHF','SDY','RSP','XLY','ITOT','IVE','SCHX','HYG','SHV','VBR','EMB','SHY','VV','SCHB','XLI','BIV','VT','MBB','BNDX','IWS','VXUS','FLOT','IWO','IXUS','MINT','SCZ','PYZ','MXI','IYM','IXP','RXI','VCR','RHS','VDC','PXI','PXE','IEO','RYF','IYG','KIE','FBT','PTH','IHI','ITA','VIS','ICF','REZ','RWR','PSJ','IGV','RYU','IDU','VPU']
data = pd.read_csv('stats_100_etfs.csv')
data

,date,open,high,low,close,volume,ticker,Clf_Target,1day_change,2day_change,3day_change,4day_change,5day_change,ewma7,ewma50,ewma200,MACD,RSI
0,2015-01-29,200.38,202.30,198.68,201.99,173585424,SPY,0.0,-1.85,0.75,3.46,2.98,4.11,202.607510,202.813380,202.823244,-0.051351,49.664254
1,2015-01-30,200.57,202.17,199.13,199.45,197729724,SPY,1.0,2.54,0.69,3.29,6.00,5.52,201.812154,202.546072,202.608549,-0.250332,44.689405
2,2015-02-02,200.05,202.03,197.86,201.92,163106969,SPY,1.0,-2.47,0.07,-1.78,0.82,3.53,201.839268,202.498240,202.566959,-0.243913,49.940725
3,2015-02-03,203.00,204.85,202.55,204.84,124212881,SPY,0.0,-2.92,-5.39,-2.85,-4.70,-2.10,202.592637,202.670737,202.697663,-0.048698,55.339046
4,2015-02-04,203.92,205.38,203.51,204.06,134306728,SPY,1.0,0.78,-2.14,-4.61,-2.07,-3.92,202.960644,202.769663,202.772444,0.051507,53.673969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123895,2019-12-23,143.28,143.29,141.21,141.55,173189,VPU,1.0,1.49,0.41,0.53,-0.23,-0.62,141.546517,140.143844,135.478711,0.616715,56.661665
123896,2019-12-24,141.59,141.78,141.04,141.62,52726,VPU,1.0,-0.07,1.42,0.34,0.46,-0.30,141.564887,140.201733,135.539819,0.617189,57.012288
123897,2019-12-26,141.85,142.05,141.37,141.86,68987,VPU,1.0,-0.24,-0.31,1.18,0.10,0.22,141.638666,140.266763,135.602706,0.629673,58.259176
123898,2019-12-27,142.01,142.27,141.71,142.25,107721,VPU,0.5,-0.39,-0.63,-0.70,0.79,-0.29,141.791499,140.344537,135.668849,0.663388,60.275586


In [13]:
df = data.loc[data['ticker'] == 'SPY']
df

,date,open,high,low,close,volume,ticker,Clf_Target,1day_change,2day_change,3day_change,4day_change,5day_change,ewma7,ewma50,ewma200,MACD,RSI
0,2015-01-29,200.38,202.30,198.68,201.99,173585424,SPY,0.0,-1.85,0.75,3.46,2.98,4.11,202.607510,202.813380,202.823244,-0.051351,49.664254
1,2015-01-30,200.57,202.17,199.13,199.45,197729724,SPY,1.0,2.54,0.69,3.29,6.00,5.52,201.812154,202.546072,202.608549,-0.250332,44.689405
2,2015-02-02,200.05,202.03,197.86,201.92,163106969,SPY,1.0,-2.47,0.07,-1.78,0.82,3.53,201.839268,202.498240,202.566959,-0.243913,49.940725
3,2015-02-03,203.00,204.85,202.55,204.84,124212881,SPY,0.0,-2.92,-5.39,-2.85,-4.70,-2.10,202.592637,202.670737,202.697663,-0.048698,55.339046
4,2015-02-04,203.92,205.38,203.51,204.06,134306728,SPY,1.0,0.78,-2.14,-4.61,-2.07,-3.92,202.960644,202.769663,202.772444,0.051507,53.673969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1234,2019-12-23,321.59,321.65,321.06,321.22,53015641,SPY,1.0,-0.49,-0.32,-1.63,-1.65,-1.72,319.588712,310.365505,296.481980,3.306089,74.296484
1235,2019-12-24,321.47,321.52,320.90,321.23,20270007,SPY,1.0,-0.01,-0.50,-0.33,-1.64,-1.66,319.999034,310.791563,296.728230,3.310214,74.315306
1236,2019-12-26,321.65,322.95,321.64,322.94,31024188,SPY,0.0,-1.71,-1.72,-2.21,-2.04,-3.35,320.734276,311.267973,296.989045,3.412134,77.367412
1237,2019-12-27,323.74,323.80,322.28,322.86,42554820,SPY,0.0,0.08,-1.63,-1.64,-2.13,-1.96,321.265707,311.722562,297.246468,3.446719,76.906976


In [5]:
window_len = 10
#Create a data point (i.e. a date) which splits the training and testing set
split_date = list(df["date"][-(2*window_len+1):])[0]

#Split the training and test set
training_set, test_set = df[df['date'] < split_date], df[df['date'] >= split_date]
training_set = training_set.drop(['date','ticker'], 1)
test_set = test_set.drop(['date','ticker'], 1)

#Create windows for training
LSTM_training_inputs = []
for i in range(len(training_set)-window_len):
    temp_set = training_set[i:(i+window_len)].copy()
    
    for col in list(temp_set):
        temp_set[col] = temp_set[col]/temp_set[col].iloc[0] - 1
    
    LSTM_training_inputs.append(temp_set)
LSTM_training_outputs = (training_set['close'][window_len:].values/training_set['close'][:-window_len].values)-1

LSTM_training_inputs = [np.array(LSTM_training_input) for LSTM_training_input in LSTM_training_inputs]
LSTM_training_inputs = np.array(LSTM_training_inputs)

#Create windows for testing
LSTM_test_inputs = []
for i in range(len(test_set)-window_len):
    temp_set = test_set[i:(i+window_len)].copy()
    
    for col in list(temp_set):
        temp_set[col] = temp_set[col]/temp_set[col].iloc[0] - 1
    
    LSTM_test_inputs.append(temp_set)
LSTM_test_outputs = (test_set['close'][window_len:].values/test_set['close'][:-window_len].values)-1

LSTM_test_inputs = [np.array(LSTM_test_inputs) for LSTM_test_inputs in LSTM_test_inputs]
LSTM_test_inputs = np.array(LSTM_test_inputs)

In [6]:
LSTM_test_inputs.shape

(11, 10, 16)

In [7]:
def build_model(inputs, output_size, neurons, activ_func="linear",
                dropout=0.40, loss="mae", optimizer="adam"):
    model = Sequential()
    model.add(LSTM(neurons, input_shape=(inputs.shape[1], inputs.shape[2])))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))
    model.compile(loss=loss, optimizer=optimizer)
    return model

In [8]:
model = build_model(LSTM_training_inputs, output_size=1, neurons = 32)
history = model.fit(LSTM_training_inputs, LSTM_training_outputs, epochs=10, batch_size=1, verbose=2, shuffle=True)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/10
 - 10s - loss: nan
Epoch 2/10
 - 9s - loss: nan
Epoch 3/10
 - 11s - loss: nan
Epoch 4/10
 - 10s - loss: nan
Epoch 5/10
 - 10s - loss: nan
Epoch 6/10
 - 9s - loss: nan
Epoch 7/10
 - 10s - loss: nan
Epoch 8/10
 - 9s - loss: nan
Epoch 9/10
 - 13s - loss: nan
Epoch 10/10
 - 13s - loss: nan


In [9]:
history.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                6272      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 6,305
Trainable params: 6,305
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.predict(test_set)

ValueError: Error when checking input: expected lstm_1_input to have 3 dimensions, but got array with shape (21, 16)